# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """

    # Returned Variables initialized & Texts converted into setences (using \n as separator)
    source_id_text = [[source_vocab_to_int[word] for word in line.split()] for line in source_text.split('\n')]
    target_id_text = [[target_vocab_to_int[word] for word in line.split()] for line in target_text.split('\n')]
    
    # Iterate sentence per sentence splitting words per sentence id FOR TARGETS
    eos = target_vocab_to_int['<EOS>']
    target_id_text = [line + [eos] for line in target_id_text]
    return source_id_text, target_id_text


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    inputs=tf.placeholder(shape=[None,None],name='input',dtype=tf.int32)
    targets=tf.placeholder(shape=[None,None],name='targets',dtype=tf.int32)
    
    learning_rate=tf.placeholder(name='learning_rate',dtype=tf.float32)
    keep_prob=tf.placeholder(name='keep_prob',dtype=tf.float32)
    
    target_seqence_length=tf.placeholder(shape=[None],name='target_sequence_length',dtype=tf.int32)
    
    max_target_len=tf.reduce_max(target_seqence_length)
    source_sequence_length=tf.placeholder(shape=[None],name='source_sequence_length',dtype=tf.int32)
    
    return inputs, targets, learning_rate, keep_prob,target_seqence_length, max_target_len, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """ 
    target_data_slice = tf.strided_slice(
        input_=target_data, # in target_data 
        begin=[0, 0], # from start point
        end=[batch_size, -1], # remove the last word on it batch
        strides=[1, 1] # each batch
    )

    # Create it GO ID
    go = tf.multiply(tf.ones([batch_size, 1], tf.int32),target_vocab_to_int['<GO>'])
    # concat the GO ID to
    # begining of each batch
    target_data = tf.concat([go, target_data_slice], 1)    
    return target_data


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """

    # Embed the encoder input using tf.contrib.layers.embed_sequence    
    embed = tf.contrib.layers.embed_sequence(rnn_inputs,vocab_size=source_vocab_size,embed_dim=encoding_embedding_size)

    # Construct a stacked tf.contrib.rnn.LSTMCell wrapped in a tf.contrib.rnn.DropoutWrapper
    def make_cell():
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        drop = tf.contrib.rnn.DropoutWrapper(cell=lstm,output_keep_prob=keep_prob)
        return drop
    
    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell() for _ in range(num_layers)])
    
    rnn_enc_output, rnn_enc_state = tf.nn.dynamic_rnn(enc_cell, embed,
                                                      sequence_length=source_sequence_length,
                                                      dtype=tf.float32)
    
    return (rnn_enc_output,rnn_enc_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # Create a tf.contrib.seq2seq.TrainingHelper
    # Create a tf.contrib.seq2seq.BasicDecoder
    # Obtain the decoder outputs from tf.contrib.seq2seq.dynamic_decode
    # Helper for the training process. Used by BasicDecoder to read inputs.
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=target_sequence_length,
                                                        time_major=False)


    # Basic decoder
    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       encoder_state,
                                                       output_layer) 

    # Perform dynamic decoding using the decoder
    training_decoder_output = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                impute_finished=True,
                                                                maximum_iterations=max_summary_length)[0]     

    
    # RETURN: BasicDecoderOutput containing training logits and sample_id 
    # (decoder outputs from tf.contrib.seq2seq.dynamic_decode)
    return training_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # Create a tf.contrib.seq2seq.GreedyEmbeddingHelper    
    embedding = dec_embeddings
    end_token = end_of_sequence_id
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_tokens')
    greedy_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embedding, start_tokens,end_token)
    #drop_dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, output_keep_prob=keep_prob)
    
    # Create a tf.contrib.seq2seq.BasicDecoder
    decoder = tf.contrib.seq2seq.BasicDecoder(
        cell=dec_cell, #cell=drop_dec_cell,
        helper=greedy_helper,
        initial_state=encoder_state, #initial_state=dec_cell.zero_state(batch_size, tf.float32))
        output_layer=output_layer)
    
    # Obtain the decoder outputs from tf.contrib.seq2seq.dynamic_decode
    outputs, _ = tf.contrib.seq2seq.dynamic_decode(
        decoder=decoder,
        impute_finished=True,
        maximum_iterations=max_target_sequence_length)
    
    return outputs


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size, num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    
    # 1. Decoder Embedding
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    # 2. Construct the decoder LSTM cell (just like you constructed the encoder cell above)
    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return dec_cell

    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
     
    # 3. Dense layer to translate the decoder's output at each time 
    # step into a choice from the target vocabulary
    # Create an output layer to map the outputs of the decoder to the elements of our vocabulary
    output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))


    # 4. Set up a training decoder and an inference decoder
    # Training Decoder using mine: 
    # decoding_layer_train function to get the training logits.
    with tf.variable_scope("decode"):
        training_decoder_output = decoding_layer_train(encoder_state, dec_cell, 
                             dec_embed_input, target_sequence_length, 
                             max_target_sequence_length, output_layer, keep_prob)

    # 5. Inference Decoder
    # Reuses the same parameters trained by the training process
    # using mine decoding_layer_infer function to get the inference logits.
    with tf.variable_scope("decode", reuse=True):
        start_of_sequence_id = target_vocab_to_int['<GO>']
        end_of_sequence_id = target_vocab_to_int['<EOS>']
        inference_decoder_output = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, 
                                                         start_of_sequence_id, end_of_sequence_id, 
                                                         max_target_sequence_length, target_vocab_size, 
                                                         output_layer, batch_size, keep_prob)         

    return training_decoder_output, inference_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """

    # Pass the input data through the encoder. We'll ignore the encoder output, but use the state
    _, enc_state = encoding_layer(input_data, 
                                  rnn_size, 
                                  num_layers,
                                  keep_prob,
                                  source_sequence_length,
                                  source_vocab_size, 
                                  enc_embedding_size)
    
    # Prepare the target sequences we'll feed to the decoder in training mode
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    # Pass encoder state and decoder inputs to the decoders
    return decoding_layer(dec_input, enc_state,
                          target_sequence_length, max_target_sentence_length,
                          rnn_size, num_layers, target_vocab_to_int, target_vocab_size,
                          batch_size, keep_prob, dec_embedding_size)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [14]:
# Number of Epochs
# Training for 60 epochs is not ideal. You should get a better trained model within 10 epochs with a good set of hyperparameters.
epochs = 10
# Batch Size
batch_size = 512 # Batch size of 128 is too small. It takes a lot of time to train the network. In order to speed up the training significantly, you could use a batch size of 256 or 512 without any noticeable performance drop.
# RNN Size
rnn_size = 128 # RNN Size is also small. You should use a larger RNN size in order to give the network enough power to learn the intricacies of the language. Please try RNN size in the range 128 - 256.
# Number of Layers
num_layers = 2
# Embedding Size
# The embedding size is too low. We have a vocabulary of 227 words. We want to choose an embedding size such that it will be able to capture the semantics of the words. An embedding size in the range 50 - 128 works quite well for this problem. I would recommend going to the higher side of this range.
encoding_embedding_size = 128
decoding_embedding_size = 128
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
# You are using no dropout. However, as I have suggested earlier, you should use dropout in the encoder cells and decoder cells (during training only). Once, you are using dropout in the model, you can try with 30% - 50% dropout.
keep_probability = 0.55
display_step = True

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/269 - Train Accuracy: 0.2329, Validation Accuracy: 0.3096, Loss: 5.7242
Epoch   0 Batch    2/269 - Train Accuracy: 0.2655, Validation Accuracy: 0.3096, Loss: 5.4961
Epoch   0 Batch    3/269 - Train Accuracy: 0.2444, Validation Accuracy: 0.3096, Loss: 5.3196
Epoch   0 Batch    4/269 - Train Accuracy: 0.2317, Validation Accuracy: 0.3096, Loss: 5.1247
Epoch   0 Batch    5/269 - Train Accuracy: 0.2324, Validation Accuracy: 0.3096, Loss: 4.9257
Epoch   0 Batch    6/269 - Train Accuracy: 0.2785, Validation Accuracy: 0.3096, Loss: 4.5882
Epoch   0 Batch    7/269 - Train Accuracy: 0.2764, Validation Accuracy: 0.3096, Loss: 4.4242
Epoch   0 Batch    8/269 - Train Accuracy: 0.2405, Validation Accuracy: 0.3096, Loss: 4.4278
Epoch   0 Batch    9/269 - Train Accuracy: 0.2674, Validation Accuracy: 0.3096, Loss: 4.1880
Epoch   0 Batch   10/269 - Train Accuracy: 0.2409, Validation Accuracy: 0.3168, Loss: 4.2295
Epoch   0 Batch   11/269 - Train Accuracy: 0.2846, Validation Accuracy

Epoch   0 Batch   89/269 - Train Accuracy: 0.4573, Validation Accuracy: 0.4785, Loss: 2.3504
Epoch   0 Batch   90/269 - Train Accuracy: 0.4204, Validation Accuracy: 0.4839, Loss: 2.4743
Epoch   0 Batch   91/269 - Train Accuracy: 0.4178, Validation Accuracy: 0.4496, Loss: 2.3372
Epoch   0 Batch   92/269 - Train Accuracy: 0.4605, Validation Accuracy: 0.4854, Loss: 2.3457
Epoch   0 Batch   93/269 - Train Accuracy: 0.4783, Validation Accuracy: 0.4834, Loss: 2.2463
Epoch   0 Batch   94/269 - Train Accuracy: 0.4355, Validation Accuracy: 0.4645, Loss: 2.3243
Epoch   0 Batch   95/269 - Train Accuracy: 0.4645, Validation Accuracy: 0.4878, Loss: 2.3224
Epoch   0 Batch   96/269 - Train Accuracy: 0.4545, Validation Accuracy: 0.4824, Loss: 2.3123
Epoch   0 Batch   97/269 - Train Accuracy: 0.4423, Validation Accuracy: 0.4712, Loss: 2.2964
Epoch   0 Batch   98/269 - Train Accuracy: 0.4776, Validation Accuracy: 0.4877, Loss: 2.2596
Epoch   0 Batch   99/269 - Train Accuracy: 0.4227, Validation Accuracy

Epoch   0 Batch  179/269 - Train Accuracy: 0.4849, Validation Accuracy: 0.5009, Loss: 1.8200
Epoch   0 Batch  180/269 - Train Accuracy: 0.4771, Validation Accuracy: 0.5047, Loss: 1.7810
Epoch   0 Batch  181/269 - Train Accuracy: 0.4541, Validation Accuracy: 0.4846, Loss: 1.7795
Epoch   0 Batch  182/269 - Train Accuracy: 0.4834, Validation Accuracy: 0.5029, Loss: 1.8044
Epoch   0 Batch  183/269 - Train Accuracy: 0.5467, Validation Accuracy: 0.5059, Loss: 1.5488
Epoch   0 Batch  184/269 - Train Accuracy: 0.4300, Validation Accuracy: 0.4852, Loss: 1.8543
Epoch   0 Batch  185/269 - Train Accuracy: 0.4953, Validation Accuracy: 0.5053, Loss: 1.7636
Epoch   0 Batch  186/269 - Train Accuracy: 0.4636, Validation Accuracy: 0.5137, Loss: 1.8312
Epoch   0 Batch  187/269 - Train Accuracy: 0.4689, Validation Accuracy: 0.4916, Loss: 1.7226
Epoch   0 Batch  188/269 - Train Accuracy: 0.4838, Validation Accuracy: 0.4966, Loss: 1.7012
Epoch   0 Batch  189/269 - Train Accuracy: 0.4881, Validation Accuracy

Epoch   0 Batch  267/269 - Train Accuracy: 0.4879, Validation Accuracy: 0.5160, Loss: 1.4466
Epoch   1 Batch    1/269 - Train Accuracy: 0.4793, Validation Accuracy: 0.5262, Loss: 1.4761
Epoch   1 Batch    2/269 - Train Accuracy: 0.4828, Validation Accuracy: 0.5201, Loss: 1.4412
Epoch   1 Batch    3/269 - Train Accuracy: 0.4590, Validation Accuracy: 0.5063, Loss: 1.4598
Epoch   1 Batch    4/269 - Train Accuracy: 0.4697, Validation Accuracy: 0.5214, Loss: 1.4664
Epoch   1 Batch    5/269 - Train Accuracy: 0.4609, Validation Accuracy: 0.5177, Loss: 1.4775
Epoch   1 Batch    6/269 - Train Accuracy: 0.4875, Validation Accuracy: 0.5041, Loss: 1.3439
Epoch   1 Batch    7/269 - Train Accuracy: 0.4959, Validation Accuracy: 0.5117, Loss: 1.3881
Epoch   1 Batch    8/269 - Train Accuracy: 0.4702, Validation Accuracy: 0.5131, Loss: 1.4529
Epoch   1 Batch    9/269 - Train Accuracy: 0.4655, Validation Accuracy: 0.5090, Loss: 1.4004
Epoch   1 Batch   10/269 - Train Accuracy: 0.4733, Validation Accuracy

Epoch   1 Batch   90/269 - Train Accuracy: 0.4865, Validation Accuracy: 0.5430, Loss: 1.2279
Epoch   1 Batch   91/269 - Train Accuracy: 0.4983, Validation Accuracy: 0.5202, Loss: 1.1463
Epoch   1 Batch   92/269 - Train Accuracy: 0.5133, Validation Accuracy: 0.5321, Loss: 1.1402
Epoch   1 Batch   93/269 - Train Accuracy: 0.5573, Validation Accuracy: 0.5526, Loss: 1.1123
Epoch   1 Batch   94/269 - Train Accuracy: 0.5040, Validation Accuracy: 0.5232, Loss: 1.1677
Epoch   1 Batch   95/269 - Train Accuracy: 0.5068, Validation Accuracy: 0.5264, Loss: 1.1477
Epoch   1 Batch   96/269 - Train Accuracy: 0.5281, Validation Accuracy: 0.5461, Loss: 1.1366
Epoch   1 Batch   97/269 - Train Accuracy: 0.5204, Validation Accuracy: 0.5385, Loss: 1.1476
Epoch   1 Batch   98/269 - Train Accuracy: 0.4875, Validation Accuracy: 0.5063, Loss: 1.1191
Epoch   1 Batch   99/269 - Train Accuracy: 0.5086, Validation Accuracy: 0.5458, Loss: 1.2046
Epoch   1 Batch  100/269 - Train Accuracy: 0.5522, Validation Accuracy

...

Epoch   8 Batch   95/269 - Train Accuracy: 0.9169, Validation Accuracy: 0.8988, Loss: 0.0833
Epoch   8 Batch   96/269 - Train Accuracy: 0.9033, Validation Accuracy: 0.9036, Loss: 0.0895
Epoch   8 Batch   97/269 - Train Accuracy: 0.9089, Validation Accuracy: 0.8963, Loss: 0.0848
Epoch   8 Batch   98/269 - Train Accuracy: 0.9199, Validation Accuracy: 0.8988, Loss: 0.0825
Epoch   8 Batch   99/269 - Train Accuracy: 0.9002, Validation Accuracy: 0.8920, Loss: 0.0852
Epoch   8 Batch  100/269 - Train Accuracy: 0.9134, Validation Accuracy: 0.8970, Loss: 0.0862
Epoch   8 Batch  101/269 - Train Accuracy: 0.9152, Validation Accuracy: 0.8999, Loss: 0.0944
Epoch   8 Batch  102/269 - Train Accuracy: 0.9145, Validation Accuracy: 0.8987, Loss: 0.0826
Epoch   8 Batch  103/269 - Train Accuracy: 0.9204, Validation Accuracy: 0.8992, Loss: 0.0889
Epoch   8 Batch  104/269 - Train Accuracy: 0.9084, Validation Accuracy: 0.9001, Loss: 0.0805
Epoch   8 Batch  105/269 - Train Accuracy: 0.9039, Validation Accuracy

Epoch   8 Batch  185/269 - Train Accuracy: 0.9250, Validation Accuracy: 0.9035, Loss: 0.0755
Epoch   8 Batch  186/269 - Train Accuracy: 0.9114, Validation Accuracy: 0.9043, Loss: 0.0727
Epoch   8 Batch  187/269 - Train Accuracy: 0.9110, Validation Accuracy: 0.8994, Loss: 0.0780
Epoch   8 Batch  188/269 - Train Accuracy: 0.9208, Validation Accuracy: 0.9001, Loss: 0.0792
Epoch   8 Batch  189/269 - Train Accuracy: 0.9274, Validation Accuracy: 0.9014, Loss: 0.0773
Epoch   8 Batch  190/269 - Train Accuracy: 0.9203, Validation Accuracy: 0.9088, Loss: 0.0751
Epoch   8 Batch  191/269 - Train Accuracy: 0.9005, Validation Accuracy: 0.9047, Loss: 0.0795
Epoch   8 Batch  192/269 - Train Accuracy: 0.9142, Validation Accuracy: 0.9015, Loss: 0.0770
Epoch   8 Batch  193/269 - Train Accuracy: 0.9228, Validation Accuracy: 0.9097, Loss: 0.0701
Epoch   8 Batch  194/269 - Train Accuracy: 0.9128, Validation Accuracy: 0.9027, Loss: 0.0833
Epoch   8 Batch  195/269 - Train Accuracy: 0.8983, Validation Accuracy

Epoch   9 Batch    7/269 - Train Accuracy: 0.9304, Validation Accuracy: 0.9196, Loss: 0.0667
Epoch   9 Batch    8/269 - Train Accuracy: 0.9321, Validation Accuracy: 0.9116, Loss: 0.0705
Epoch   9 Batch    9/269 - Train Accuracy: 0.9182, Validation Accuracy: 0.9132, Loss: 0.0729
Epoch   9 Batch   10/269 - Train Accuracy: 0.9199, Validation Accuracy: 0.9103, Loss: 0.0633
Epoch   9 Batch   11/269 - Train Accuracy: 0.9302, Validation Accuracy: 0.9103, Loss: 0.0737
Epoch   9 Batch   12/269 - Train Accuracy: 0.9054, Validation Accuracy: 0.9067, Loss: 0.0785
Epoch   9 Batch   13/269 - Train Accuracy: 0.9292, Validation Accuracy: 0.9080, Loss: 0.0579
Epoch   9 Batch   14/269 - Train Accuracy: 0.9257, Validation Accuracy: 0.9159, Loss: 0.0677
Epoch   9 Batch   15/269 - Train Accuracy: 0.9292, Validation Accuracy: 0.9142, Loss: 0.0568
Epoch   9 Batch   16/269 - Train Accuracy: 0.9163, Validation Accuracy: 0.9141, Loss: 0.0714
Epoch   9 Batch   17/269 - Train Accuracy: 0.9201, Validation Accuracy

Epoch   9 Batch   96/269 - Train Accuracy: 0.9118, Validation Accuracy: 0.9162, Loss: 0.0664
Epoch   9 Batch   97/269 - Train Accuracy: 0.9309, Validation Accuracy: 0.9194, Loss: 0.0655
Epoch   9 Batch   98/269 - Train Accuracy: 0.9385, Validation Accuracy: 0.9144, Loss: 0.0620
Epoch   9 Batch   99/269 - Train Accuracy: 0.9137, Validation Accuracy: 0.9171, Loss: 0.0633
Epoch   9 Batch  100/269 - Train Accuracy: 0.9358, Validation Accuracy: 0.9177, Loss: 0.0656
Epoch   9 Batch  101/269 - Train Accuracy: 0.9301, Validation Accuracy: 0.9147, Loss: 0.0724
Epoch   9 Batch  102/269 - Train Accuracy: 0.9288, Validation Accuracy: 0.9118, Loss: 0.0616
Epoch   9 Batch  103/269 - Train Accuracy: 0.9302, Validation Accuracy: 0.9145, Loss: 0.0693
Epoch   9 Batch  104/269 - Train Accuracy: 0.9259, Validation Accuracy: 0.9154, Loss: 0.0648
Epoch   9 Batch  105/269 - Train Accuracy: 0.9122, Validation Accuracy: 0.9204, Loss: 0.0637
Epoch   9 Batch  106/269 - Train Accuracy: 0.9335, Validation Accuracy

Epoch   9 Batch  185/269 - Train Accuracy: 0.9363, Validation Accuracy: 0.9174, Loss: 0.0567
Epoch   9 Batch  186/269 - Train Accuracy: 0.9276, Validation Accuracy: 0.9213, Loss: 0.0558
Epoch   9 Batch  187/269 - Train Accuracy: 0.9321, Validation Accuracy: 0.9217, Loss: 0.0552
Epoch   9 Batch  188/269 - Train Accuracy: 0.9343, Validation Accuracy: 0.9198, Loss: 0.0549
Epoch   9 Batch  189/269 - Train Accuracy: 0.9338, Validation Accuracy: 0.9190, Loss: 0.0558
Epoch   9 Batch  190/269 - Train Accuracy: 0.9319, Validation Accuracy: 0.9175, Loss: 0.0551
Epoch   9 Batch  191/269 - Train Accuracy: 0.9195, Validation Accuracy: 0.9262, Loss: 0.0565
Epoch   9 Batch  192/269 - Train Accuracy: 0.9302, Validation Accuracy: 0.9293, Loss: 0.0560
Epoch   9 Batch  193/269 - Train Accuracy: 0.9361, Validation Accuracy: 0.9214, Loss: 0.0536
Epoch   9 Batch  194/269 - Train Accuracy: 0.9203, Validation Accuracy: 0.9207, Loss: 0.0595
Epoch   9 Batch  195/269 - Train Accuracy: 0.9167, Validation Accuracy

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [20]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # Downcase the sentence
    sentence = sentence.lower()
    # Convert words into ids & add replace "not-words" to <UNK>
    list_words=[vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in sentence.split(" ")]
    return list_words


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [21]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [10, 29, 158, 84, 60, 61, 105]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [243, 46, 52, 179, 39, 40, 1]
  French Words: il vu un vieux camion . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.